In [1]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)
import nest_asyncio
nest_asyncio.apply()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer


# Set LLM (OpenAI)

In [2]:
#load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [3]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [ ]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Set LLM for chat  (Local)

In [36]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

# Load storage contexts

## Load vector storage context

In [ ]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [39]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [ ]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)
print(onto_storage_context)

# Load index

## Load vector index

In [ ]:
simple_index = load_index_from_storage(vector_storage_context)

### Set retriever

In [43]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [ ]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan.",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

## Load graph index

In [ ]:
graph_index = load_index_from_storage(graph_storage_context)

In [47]:
nx_graph = graph_index.get_networkx_graph()

In [ ]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

In [ ]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

### Set retriever

In [50]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [ ]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan.",
)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

## Load onto graph index

In [ ]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

In [8]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set onto graph retriever

In [9]:
onto_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True,
    top_k=20,
    depth=3 
)

onto_engine = RetrieverQueryEngine.from_args(
    onto_retriever,
    include_text=True
)


### Test LLM based transparent retriever

In [ ]:
response = onto_engine.query(
    "I build a ML system, what does it imply in terms of data governance ?",
)

# Access the source nodes
for node in response.source_nodes:
    print(node.node.metadata)
    # Access node content
    print("**Content**")
    content = node.node.get_content()
    print(content)
    
    # Access node score
    print("**Score**")
    score = node.score
    print(score)
    print("--"*20)

display(Markdown(f"<b>{response}</b>"))

# Understand knowledge graph

In [ ]:
# Get all nodes
all_nodes = onto_storage_context.property_graph_store.get()

# Get the relation map for all nodes
rel_map = onto_storage_context.property_graph_store.get_rel_map(graph_nodes=all_nodes, limit=400)


# print only relations 
for rel in rel_map:
    print(rel[1])
    


In [ ]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

# (Simple) Query the vectors

In [ ]:
query = "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan."
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

# (Simple) Query the knowledge graph 

In [ ]:
query = "Quelle méthode utiliser pour prédire si un client va faire défaut sur son prêt bancaire. Fais moi un plan"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

# (Simple) Query the onto graph 

In [ ]:
query = "Explique moi le théorème de Bayes"
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=10,
)

response = onto_query_engine.query(query)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

## (Node retriever)

In [ ]:
retriever = onto_index.as_retriever(
    include_text=False,  # include source text, default True
)

nodes = retriever.retrieve("Qu'est-ce que le machine learning ?")


for node in nodes:
    print(node.text)


# Have a real chat with your data

## Set up the engines

### Vector engines

In [68]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        " "
        " "
        "."
    ),
    verbose=False,
)

### Graph engines

In [69]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [15]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
onto_chat_engine = onto_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        ""
        " "
        "."
    ),
    verbose=False,
)

## Generate

In [24]:
chat_engine.reset()

In [25]:
memory = ChatMemoryBuffer.from_defaults(token_limit=10000)
chat_engine = onto_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu écris une newsletter qui s'appelle le bateau ivre des données. Une newsletter pour apprendre à naviguer et concevoir des dispositifs dans les mers étranges des données"
        "Dans cette newsletter il est question d'exposer des outils, des réflexions ou des ouvrages mais plutôt de vous livrer, chaque semaine, le fruit d'une investigation au coeur d'un phénomène étrange rencontré lors d'interactions homme-données. "
        """ Chaque article se construit en 5 parties : 1️. le contexte d'émergence du phénomène, 2. l'évènement qui donne envie d'explorer le phénomène 3. les outils qui permettent d'explorer le phénomène 4.l'origine ou le décryptage de la sensation d'étrangeté 5. les choix, solutions et outils possibles pour solutionner cette étrangeté"""
    ),
    verbose=False,
)

## Produce content

In [ ]:
response_stream = chat_engine.stream_chat("""
                                                Tu vas m'écrire l'introduction d'un article du bateau ivre des données qui vise à decrypter l'étrange fait que les modèles de machine learning n'ont parfois pas le comportement attendu.  
                                                Cet article vise à decrypter les phénomènes de concept drift, data drift, semantic drift mais aussi les émergences de systèmes complexes
                                                L'article vise à répondre à cette première question : pourquoi des modèles de machine learning ont des comportements non-prévus par leurs concepteurs ? 
                                                Puis à cette deuxième question : comment peut-on réinterpréter les résultats d'un modèle de machine learning une fois ceux-ci produits ? comment faire une forme de rétro-ingénierie de la décision du modèle ?
                                                Et enfin à cette troisième question : comment peut-on se servir de ces résultats pour améliorer le modèle ? 
                                                """)

In [ ]:
generate = response_stream.print_response_stream()

In [ ]:
response_stream = chat_engine.stream_chat("""
                                                Génère la première section de l'article
                                                """)

In [ ]:
generate = response_stream.print_response_stream()

In [ ]:
for token in response_stream.response_gen:
    print(token, end="")

retrieved_nodes = response_stream.source_nodes


for node in retrieved_nodes:
    print(node.text)
    print("-" * 40)


In [ ]:
response_stream = chat_engine.stream_chat("""
                                                Génère la deuxième section de l'article.
                                                """)

In [ ]:
generate = response_stream.print_response_stream()

In [ ]:
response_stream = chat_engine.stream_chat("""
                                                Génère la troisième section de l'article.
                                                """)

In [ ]:
generate = response_stream.print_response_stream()

In [ ]:
response_stream = chat_engine.stream_chat("""
                                                Génère la quatrième section de l'article.
                                                """)

In [ ]:
generate = response_stream.print_response_stream()

In [ ]:
response_stream = chat_engine.stream_chat("""
                                                Génère la cinquième section de l'article en évitant les bullets points.
                                                """)

In [ ]:
generate = response_stream.print_response_stream()

In [ ]:
response_stream = chat_engine.stream_chat("""
                                                Génère une conclusion pour l'article qui répond à l'ensemble des questions posées initialement.
                                                
                                                """)

In [ ]:
generate = response_stream.print_response_stream()

### Sum-up

In [40]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"/Users/arthursarazin/Documents/knowledge_glossary/output.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 

## Answer question

In [ ]:
# Supposons que chat_engine soit déjà défini avec ton LlamaIndex ou un moteur similaire

# 📝 Liste des prompts

prompts = [

    "What are the best ways to transform data for different audiences?",
    "What are your best data quality risk management strategies?",
    "How do you improve data relevance for your audience?",
    "How can you justify data quality investments?",
    "What are the best ways to learn and share data quality practices?",
    "How do you choose a data quality vendor?",
    "What is your strategy for incorporating data quality dimensions and attributes?"

]


# 📝 Fonction pour générer les réponses
def stream_responses(prompts, chat_engine):
    """Itère sur une liste de prompts et affiche les réponses générées."""
    for i, prompt in enumerate(prompts, start=1):
        print(f"\n🔹 **Prompt {i}:** {prompt}\n")

        # 🚀 Appelle stream_chat avec le prompt
        response_stream = chat_engine.stream_chat(
            f"{prompt} Answer with one paragraph in less than 600 characters and use the first person (I...) "
            "Use your knowledge base to provide a detailed response."
        
        )

        # 📝 Collecte la réponse
        generate = response_stream.print_response_stream()
        
        

# 🚀 Exécuter la fonction
stream_responses(prompts, chat_engine)


## Sum-up